In [ ]:
import pandas as pd
import numpy as np

from src.candidate_generation import CandidateGenerationClassification

### Main code

In [ ]:
# Instantiate a CandidateGeneration object, which requires to pass the dataframes of (1) the labels given to the users and (2) some mapping
# between the cells of a grid and the users.
path_users_classification = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.classified.parquet'
path_users_cells_mapping = './mapping_users_cells_grid_50m.pkl'


cand_gen = CandidateGenerationClassification(path_users_classification, path_users_cells_mapping)
print(f"Global positive rate: {cand_gen.get_global_positive_rate()}")

### Various operations

In [ ]:
cnt_threshold = 1
eps_threshold = 0.05
list_candidates_tocheck = cand_gen.candidate_generation(cnt_threshold, eps_threshold)

list_candidates_tocheck